# SciPy
## Author: Gustavo Amarante
SciPy is a collection of mathematical algorithms and convenience functions. In this this notebook there are just a few examples of the features that are most important to us. But if you want to see all that SciPy has to offer, have a look at the [official documentation](https://docs.scipy.org/doc/scipy/reference/).

Since SciPy has several subpackages, it is commom practice to import just the one we are going to use, as you'll in the following examples.

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt

## Definite Integrals
The function `quad` is provided to integrate a function of one variable between two points. This functions has 2 outputs, the first one is the computed integral value and the second is an estimate of the absolute error.

In [ ]:
import scipy.integrate as integrate

def my_func(x):
    return x**2

integrate.quad(my_func, 0, 2)

The `quad` functions also allows for infinite limits.

$$
\int_{-\infty}^{\infty} e^{-x^{2}}dx
$$

In [ ]:
def my_func(x):
    return np.exp(-x**2)

integrate.quad(my_func, -np.inf, np.inf)

SciPy's `integrate` library also has functions for double and triple integrals. Check them out in the documentations.

## Optimization
The `scipy.optimize` package provides several commonly used optimization algorithms. Here we are going to use just one to illustrate.

Consider that you have 3 assets available. Their expected returns, risks (standard-deviations) and betas are on the table bellow and $\rho$ is the correlation matrix of the returns.

| Asset | Return | Risk | Beta |
|-------|--------|------|------|
|A      |3%      | 10%  | 0.5  |
|B      |3.5%    | 11%  | 1.2  |
|C      |5%      | 15%  | 1.8  |

$$
\rho = 
\begin{bmatrix}
1 & 0.3 & -0.6 \\
0.3 & 1 & 0 \\
-0.6 & 0 & 1 
\end{bmatrix}
$$

Use the `minimize` function to find the weights of each asset that maximizes it's Sharpe index.

In [ ]:
retu = np.array([0.03, 0.035, 0.05])
risk = np.array([0.10, 0.11, 0.15])
beta = np.array([0.5, 1.2, 1.8])

corr = np.array([[1, 0.3, -0.6], 
                 [0.3, 1, 0],
                 [-0.6, 0, 1]])

def port_return(w):
    return retu.dot(w)

def port_risk(w):
    covar = np.diag(risk).dot(corr).dot(np.diag(risk))
    return (w.dot(covar).dot(w))**0.5

def port_sharpe(w):
    return -1*(port_return(w) / port_risk(w))   # The -1 is because we want to MINIMIZE the negative of the Sharpe

def port_weight(w):
    return w.sum()

In [ ]:
from scipy.optimize import minimize

eq_cons = {'type': 'eq',
           'fun' : lambda w: port_weight(w) - 1}

w0 = np.array([1, 0, 0])

res = minimize(port_sharpe, w0, method='SLSQP', constraints=eq_cons, options={'ftol': 1e-9, 'disp': True})

In [ ]:
res.x

In [ ]:
res.x.sum()

In [ ]:
-1*res.fun

## Interpolation
There are several general interpolation facilities available in SciPy, for data in 1, 2, and higher dimensions. The `interp1d` funtions grabs data points and returns a function. The default interpolation method is the linear interpolation, but there are several to choose from.

In [ ]:
from scipy.interpolate import interp1d

x = np.linspace(0, 10, num=11, endpoint=True)
y = np.cos(-x**2/9.0)

In [ ]:
f1 = interp1d(x, y)  # linear is the default
f2 = interp1d(x, y, kind='cubic')  # cubic splines
f3 = interp1d(x, y, kind='nearest')  # grab the nearest value
f4 = interp1d(x, y, kind='previous')  # hold last value
f5 = interp1d(x, y, kind='next')  # grab the next value

Now that we have the interpolated function, lets generate more points for the x axis and plot the different methods

In [ ]:
xnew = np.linspace(0, 10, num=41, endpoint=True)
xnew

In [ ]:
plt.plot(x, y, 'o', xnew, f1(xnew), '-', xnew, f2(xnew), '--')
plt.legend(['data', 'linear', 'cubic'], loc='best')
plt.show()

In [ ]:
plt.plot(x, y, 'o', xnew, f3(xnew), '-', xnew, f4(xnew), '--', xnew, f5(xnew), ':')
plt.legend(['data', 'nearest', 'previous', 'next'], loc='best')
plt.show()

The `interpolate` sublibrary also has interpolation methods for multivariate data and has **integration with pandas**. Have a look at the documentation.

## Linear Algebra (again)
`scipy.linalg` contains all the functions in `numpy.linalg` plus some more advanced ones.

In [ ]:
from scipy import linalg as la

A = np.array([[1,3,5],[2,5,1],[2,3,8]])
la.inv(A)

Matrix and vector **norms** can also be computed with SciPy. A wide range of norm definitions are available using different parameters to the order argument of `linalg.norm`.

In [ ]:
A = np.array([[1, 2], [3, 4]])
print(la.norm(A))  # frobenius norm is the default.
print(la.norm(A, 1)) # L1 norm (max column sum)
print(la.norm(A, np.inf)) # L inf norm (max row sum)

Some more advanced matrix decompositions are also available, like the **Schur Decomposition**

In [ ]:
la.schur(A)

Some notable matrices can also be created, like block **diagonal matrices**.

In [ ]:
A = np.array([[1, 0],
              [0, 1]])

B = np.array([[3, 4, 5],
              [6, 7, 8]])

C = np.array([[7]])

la.block_diag(A, B, C)

## Solving Linear Systems


$$
\begin{align}
x+3y+5 & =10\\
2x+5y+z & =8\\
2x+3y+8z & =3
\end{align}
$$

The system above can be written with matrix notation as $AX=B$ and we know we can find the solution by doing $X=A^{-1}B$, but inverting a matrix is computationally expensive. When solving big linear system it is advised to use the `solve` method.

In [ ]:
A = np.array([[1, 3, 5], [2, 5, 1], [2, 3, 8]])
B = np.array([[10], [8], [3]])

Lets check the time that it takes to solve the system in both ways...

In [ ]:
la.inv(A).dot(B)

In [ ]:
la.solve(A, B)

let's try with a bigger matrix

In [51]:
import numpy.random as rnd
A = rnd.random((1000, 1000))
B = rnd.random((1000, 1))

In [66]:
%%timeit
la.inv(A).dot(B)

25.5 ms ± 350 µs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [65]:
%%time
la.solve(A, B)

Wall time: 32.9 ms


array([[ 2.31214459e-01],
       [ 4.79237034e-01],
       [ 9.53224510e-01],
       [ 3.40474383e-01],
       [ 6.05903567e-01],
       [-5.02169343e-02],
       [ 1.30073354e+00],
       [ 3.58581460e-02],
       [-1.45558914e-02],
       [ 1.00257224e+00],
       [ 2.05028738e-01],
       [-2.24503346e-02],
       [-3.60374885e-01],
       [ 2.25047802e-01],
       [-4.80140942e-01],
       [-6.91832635e-01],
       [ 8.44473901e-01],
       [ 1.42567710e-01],
       [-5.41918734e-02],
       [ 2.70487149e-01],
       [ 2.35105478e-01],
       [-2.23445599e-01],
       [-8.13089980e-01],
       [-3.69475539e-03],
       [ 7.09279202e-01],
       [ 3.39461931e-01],
       [ 9.23217507e-01],
       [-1.09690532e-01],
       [ 7.03497236e-02],
       [ 2.08463952e-01],
       [-1.15337255e-01],
       [-3.12719804e-01],
       [-8.19449652e-02],
       [ 2.80759098e-01],
       [-6.97008998e-01],
       [ 6.29683241e-01],
       [-2.14869341e-01],
       [ 1.09850487e+00],
       [ 2.3